In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

2024-12-04 21:37:07.914686: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
file_path = "ibm.csv"
file_cleaned_path = "cleaned_ibm.csv"
data = pd.read_csv(file_path)

print(data.head())

                        Date      Open      High       Low     Close  Volume  \
0  1962-01-02 00:00:00-05:00  1.518550  1.518550  1.501487  1.501487  407940   
1  1962-01-03 00:00:00-05:00  1.501487  1.514612  1.501487  1.514612  305955   
2  1962-01-04 00:00:00-05:00  1.514613  1.514613  1.498863  1.499519  274575   
3  1962-01-05 00:00:00-05:00  1.497551  1.497551  1.467363  1.469988  384405   
4  1962-01-08 00:00:00-05:00  1.468675  1.468675  1.430613  1.442425  572685   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


In [3]:
data_filtered = data[["Date", "Open", "High", "Low", "Volume", "Close"]].sort_values(by="Date")
data_cleaned = data_filtered.dropna()
print(data_filtered.head())

                        Date      Open      High       Low  Volume     Close
0  1962-01-02 00:00:00-05:00  1.518550  1.518550  1.501487  407940  1.501487
1  1962-01-03 00:00:00-05:00  1.501487  1.514612  1.501487  305955  1.514612
2  1962-01-04 00:00:00-05:00  1.514613  1.514613  1.498863  274575  1.499519
3  1962-01-05 00:00:00-05:00  1.497551  1.497551  1.467363  384405  1.469988
4  1962-01-08 00:00:00-05:00  1.468675  1.468675  1.430613  572685  1.442425


In [4]:
data_cleaned.to_csv(file_cleaned_path, index=False)

In [5]:
data = pd.read_csv(file_cleaned_path)
data.head()

,Date,Open,High,Low,Volume,Close
0,1962-01-02 00:00:00-05:00,1.518550,1.518550,1.501487,407940,1.501487
1,1962-01-03 00:00:00-05:00,1.501487,1.514612,1.501487,305955,1.514612
2,1962-01-04 00:00:00-05:00,1.514613,1.514613,1.498863,274575,1.499519
3,1962-01-05 00:00:00-05:00,1.497551,1.497551,1.467363,384405,1.469988
4,1962-01-08 00:00:00-05:00,1.468675,1.468675,1.430613,572685,1.442425


In [6]:
data["Target"] = data["Close"].shift(-1)
data.head()

,Date,Open,High,Low,Volume,Close,Target
0,1962-01-02 00:00:00-05:00,1.518550,1.518550,1.501487,407940,1.501487,1.514612
1,1962-01-03 00:00:00-05:00,1.501487,1.514612,1.501487,305955,1.514612,1.499519
2,1962-01-04 00:00:00-05:00,1.514613,1.514613,1.498863,274575,1.499519,1.469988
3,1962-01-05 00:00:00-05:00,1.497551,1.497551,1.467363,384405,1.469988,1.442425
4,1962-01-08 00:00:00-05:00,1.468675,1.468675,1.430613,572685,1.442425,1.459488


In [7]:
final_data_row = data.tail(1)
data.drop(data.tail(1).index, inplace=True)
data.dropna(inplace=True)
final_data_row

,Date,Open,High,Low,Volume,Close,Target
15825,2024-11-13 00:00:00-05:00,209.5,211.410004,209.070099,2186158,210.669998,NaN


In [8]:
# plotting correlation
data.iloc[:, 1:].corr()['Target']

Open      0.999714
High      0.999765
Low       0.999767
Volume    0.133558
Close     0.999812
Target    1.000000
Name: Target, dtype: float64

In [9]:
# Features scaling
model_features = data.drop("Target", axis=1).drop("Date", axis=1)
model_target = data["Target"]

model_feature_scaler = MinMaxScaler()
model_feature_scaler.fit(model_features)
model_scaled_features = pd.DataFrame(model_feature_scaler.transform(model_features), columns=model_features.columns.tolist())

model_target_scaler = MinMaxScaler()
model_target_scaler.fit(model_target.values.reshape(-1,1))
model_scaled_target = pd.DataFrame(model_target_scaler.transform(model_target.values.reshape(-1,1)), columns=["Target"])

In [15]:
def create_dataset(X_data, y_data, time_steps):
    X, y = [], []
    for i in range(len(X_data) - time_steps):
        v = X_data.iloc[i:(i + time_steps)].values
        X.append(v)
        y.append(y_data.iloc[i + time_steps])
    return np.array(X), np.array(y)

In [18]:
import random
import numpy as np
import pyswarm
from pyswarm import pso
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

param_ranges = {
    "lstm_units": (32, 256),           # Range for LSTM units
    "batch_size": (16, 128),           # Range for batch size
    "time_steps": (30, 120),           # Range for sequence length
    "early_stopping_patience": (5, 20),  # Range for early stopping patience
    "reduce_lr_factor": (0.1, 0.5),      # Range for reduce_lr factor
    "reduce_lr_patience": (2, 10),       # Range for reduce_lr patience
    "reduce_lr_min_lr": (1e-6, 1e-4)     # Range for minimum learning rate
}

bounds = [
    param_ranges["lstm_units"],            # Bounds for LSTM units
    param_ranges["batch_size"],            # Bounds for batch size
    param_ranges["time_steps"],            # Bounds for sequence length
    param_ranges["early_stopping_patience"],  # Bounds for early stopping patience
    param_ranges["reduce_lr_factor"],         # Bounds for reduce_lr factor
    param_ranges["reduce_lr_patience"],       # Bounds for reduce_lr patience
    param_ranges["reduce_lr_min_lr"]          # Bounds for reduce_lr min_lr
]

def objective_function(params):
    lstm_units = int(params[0])
    batch_size = int(params[1])
    time_steps = int(params[2])
    early_stopping_patience = int(params[3])
    reduce_lr_factor = float(params[4])
    reduce_lr_patience = int(params[5])
    reduce_lr_min_lr = float(params[6])
    
    print(f"\nTrying Hyperparameters: LSTM Units = {lstm_units}, Batch Size = {batch_size}, "
          f"Time Steps = {time_steps}, Early Stopping Patience = {early_stopping_patience}, "
          f"Reduce LR Factor = {reduce_lr_factor:.2f}, Reduce LR Patience = {reduce_lr_patience}, "
          f"Min LR = {reduce_lr_min_lr:.6f}")

    # Updating sequence length for data creation
    global X_train, y_train, X_test, y_test
    X, y = create_dataset(model_scaled_features, model_scaled_target, time_steps)
    train_size = int(0.80 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Converting to tensors
    X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
    X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
    y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)
    
    input_shape = (time_steps, X_train.shape[2])
    print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

    # Defining the new model architecture
    model = Sequential([
        # First LSTM layer with regularization
        LSTM(lstm_units, return_sequences=True,
             kernel_regularizer=l2(0.01),
             recurrent_regularizer=l2(0.01),
             input_shape=input_shape),
        Dropout(0.2),

        # Second LSTM layer
        LSTM(lstm_units, return_sequences=False,
             kernel_regularizer=l2(0.01),
             recurrent_regularizer=l2(0.01)),
        Dropout(0.2),

        # Output layer
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=early_stopping_patience,
        restore_best_weights=True
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=reduce_lr_factor,
        patience=reduce_lr_patience,
        min_lr=reduce_lr_min_lr
    )

    # Training the model
    history = model.fit(
        X_train_tensor, y_train_tensor,
        epochs=10,
        batch_size=batch_size,
        validation_data=(X_test_tensor, y_test_tensor),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Getting validation loss
    val_loss = min(history.history['val_loss'])

    print(f"Validation Loss: {val_loss:.6f}")

    return val_loss

# Running PSO with the new model
best_params, best_loss = pso(
    objective_function,
    lb=[b[0] for b in bounds],  # Lower bounds
    ub=[b[1] for b in bounds],  # Upper bounds
    swarmsize=5,               # Number of particles
    maxiter=5,                 # Number of iterations
    debug=True                  # Enable logging
)

print(f"\nBest Hyperparameters: LSTM Units = {int(best_params[0])}, Batch Size = {int(best_params[1])}, "
      f"Time Steps = {int(best_params[2])}, Early Stopping Patience = {int(best_params[3])}, "
      f"Reduce LR Factor = {best_params[4]:.2f}, Reduce LR Patience = {int(best_params[5])}, "
      f"Min LR = {best_params[6]:.6f}")
print(f"Best Validation Loss = {best_loss:.6f}")

No constraints given.

Trying Hyperparameters: LSTM Units = 146, Batch Size = 74, Time Steps = 87, Early Stopping Patience = 12, Reduce LR Factor = 0.46, Reduce LR Patience = 9, Min LR = 0.000070
X_train shape: (12590, 87, 5), X_test shape: (3148, 87, 5)
Epoch 1/10


2024-12-04 21:40:19.357236: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2024-12-04 21:40:19.482394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-12-04 21:40:19.482818: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f0fd0042720 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-04 21:40:19.482832: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A30, Compute Capability 8.0
2024-12-04 21:40:19.487497: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-04 21:40:19.607030: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


171/171 [==============================] - 7s 14ms/step - loss: 0.8095 - val_loss: 0.0071 - lr: 0.0010
Epoch 2/10
171/171 [==============================] - 2s 10ms/step - loss: 0.0027 - val_loss: 0.0032 - lr: 0.0010
Epoch 3/10
171/171 [==============================] - 2s 10ms/step - loss: 0.0012 - val_loss: 0.0021 - lr: 0.0010
Epoch 4/10
171/171 [==============================] - 2s 10ms/step - loss: 9.8997e-04 - val_loss: 0.0029 - lr: 0.0010
Epoch 5/10
171/171 [==============================] - 2s 10ms/step - loss: 8.5326e-04 - val_loss: 0.0029 - lr: 0.0010
Epoch 6/10
171/171 [==============================] - 2s 10ms/step - loss: 7.5168e-04 - val_loss: 0.0024 - lr: 0.0010
Epoch 7/10
171/171 [==============================] - 2s 10ms/step - loss: 7.2520e-04 - val_loss: 0.0030 - lr: 0.0010
Epoch 8/10
171/171 [==============================] - 2s 10ms/step - loss: 6.2938e-04 - val_loss: 0.0037 - lr: 0.0010
Epoch 9/10
171/171 [==============================] - 2s 10ms/step - loss: 5.82

## PSO 6, 6

In [19]:
import random
import numpy as np
import pyswarm
from pyswarm import pso
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

param_ranges = {
    "lstm_units": (32, 256),           # Range for LSTM units
    "batch_size": (16, 128),           # Range for batch size
    "time_steps": (30, 120),           # Range for sequence length
    "early_stopping_patience": (5, 20),  # Range for early stopping patience
    "reduce_lr_factor": (0.1, 0.5),      # Range for reduce_lr factor
    "reduce_lr_patience": (2, 10),       # Range for reduce_lr patience
    "reduce_lr_min_lr": (1e-6, 1e-4)     # Range for minimum learning rate
}

bounds = [
    param_ranges["lstm_units"],            # Bounds for LSTM units
    param_ranges["batch_size"],            # Bounds for batch size
    param_ranges["time_steps"],            # Bounds for sequence length
    param_ranges["early_stopping_patience"],  # Bounds for early stopping patience
    param_ranges["reduce_lr_factor"],         # Bounds for reduce_lr factor
    param_ranges["reduce_lr_patience"],       # Bounds for reduce_lr patience
    param_ranges["reduce_lr_min_lr"]          # Bounds for reduce_lr min_lr
]

def objective_function(params):
    lstm_units = int(params[0])
    batch_size = int(params[1])
    time_steps = int(params[2])
    early_stopping_patience = int(params[3])
    reduce_lr_factor = float(params[4])
    reduce_lr_patience = int(params[5])
    reduce_lr_min_lr = float(params[6])
    
    print(f"\nTrying Hyperparameters: LSTM Units = {lstm_units}, Batch Size = {batch_size}, "
          f"Time Steps = {time_steps}, Early Stopping Patience = {early_stopping_patience}, "
          f"Reduce LR Factor = {reduce_lr_factor:.2f}, Reduce LR Patience = {reduce_lr_patience}, "
          f"Min LR = {reduce_lr_min_lr:.6f}")

    # Updating sequence length for data creation
    global X_train, y_train, X_test, y_test
    X, y = create_dataset(model_scaled_features, model_scaled_target, time_steps)
    train_size = int(0.80 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Converting to tensors
    X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
    X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
    y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)
    
    input_shape = (time_steps, X_train.shape[2])
    print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

    # Defining the new model architecture
    model = Sequential([
        # First LSTM layer with regularization
        LSTM(lstm_units, return_sequences=True,
             kernel_regularizer=l2(0.01),
             recurrent_regularizer=l2(0.01),
             input_shape=input_shape),
        Dropout(0.2),

        # Second LSTM layer
        LSTM(lstm_units, return_sequences=False,
             kernel_regularizer=l2(0.01),
             recurrent_regularizer=l2(0.01)),
        Dropout(0.2),

        # Output layer
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=early_stopping_patience,
        restore_best_weights=True
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=reduce_lr_factor,
        patience=reduce_lr_patience,
        min_lr=reduce_lr_min_lr
    )

    # Training the model
    history = model.fit(
        X_train_tensor, y_train_tensor,
        epochs=10,
        batch_size=batch_size,
        validation_data=(X_test_tensor, y_test_tensor),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Getting validation loss
    val_loss = min(history.history['val_loss'])

    print(f"Validation Loss: {val_loss:.6f}")

    return val_loss

# Running PSO with the new model
best_params, best_loss = pso(
    objective_function,
    lb=[b[0] for b in bounds],  # Lower bounds
    ub=[b[1] for b in bounds],  # Upper bounds
    swarmsize=6,               # Number of particles
    maxiter=6,                 # Number of iterations
    debug=True                  # Enable logging
)

print(f"\nBest Hyperparameters: LSTM Units = {int(best_params[0])}, Batch Size = {int(best_params[1])}, "
      f"Time Steps = {int(best_params[2])}, Early Stopping Patience = {int(best_params[3])}, "
      f"Reduce LR Factor = {best_params[4]:.2f}, Reduce LR Patience = {int(best_params[5])}, "
      f"Min LR = {best_params[6]:.6f}")
print(f"Best Validation Loss = {best_loss:.6f}")

No constraints given.

Trying Hyperparameters: LSTM Units = 254, Batch Size = 28, Time Steps = 113, Early Stopping Patience = 19, Reduce LR Factor = 0.11, Reduce LR Patience = 5, Min LR = 0.000065
X_train shape: (12569, 113, 5), X_test shape: (3143, 113, 5)
Epoch 1/10
449/449 [==============================] - 9s 13ms/step - loss: 0.3975 - val_loss: 0.0061 - lr: 0.0010
Epoch 2/10
449/449 [==============================] - 5s 12ms/step - loss: 0.0011 - val_loss: 0.0030 - lr: 0.0010
Epoch 3/10
449/449 [==============================] - 5s 12ms/step - loss: 8.5507e-04 - val_loss: 0.0014 - lr: 0.0010
Epoch 4/10
449/449 [==============================] - 5s 12ms/step - loss: 7.9769e-04 - val_loss: 0.0065 - lr: 0.0010
Epoch 5/10
449/449 [==============================] - 5s 12ms/step - loss: 7.0095e-04 - val_loss: 0.0062 - lr: 0.0010
Epoch 6/10
449/449 [==============================] - 5s 12ms/step - loss: 6.2897e-04 - val_loss: 0.0012 - lr: 0.0010
Epoch 7/10
449/449 [======================

In [ ]:
PSO (7,7)

In [20]:
import random
import numpy as np
import pyswarm
from pyswarm import pso
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

param_ranges = {
    "lstm_units": (32, 256),           # Range for LSTM units
    "batch_size": (16, 128),           # Range for batch size
    "time_steps": (30, 120),           # Range for sequence length
    "early_stopping_patience": (5, 20),  # Range for early stopping patience
    "reduce_lr_factor": (0.1, 0.5),      # Range for reduce_lr factor
    "reduce_lr_patience": (2, 10),       # Range for reduce_lr patience
    "reduce_lr_min_lr": (1e-6, 1e-4)     # Range for minimum learning rate
}

bounds = [
    param_ranges["lstm_units"],            # Bounds for LSTM units
    param_ranges["batch_size"],            # Bounds for batch size
    param_ranges["time_steps"],            # Bounds for sequence length
    param_ranges["early_stopping_patience"],  # Bounds for early stopping patience
    param_ranges["reduce_lr_factor"],         # Bounds for reduce_lr factor
    param_ranges["reduce_lr_patience"],       # Bounds for reduce_lr patience
    param_ranges["reduce_lr_min_lr"]          # Bounds for reduce_lr min_lr
]

def objective_function(params):
    lstm_units = int(params[0])
    batch_size = int(params[1])
    time_steps = int(params[2])
    early_stopping_patience = int(params[3])
    reduce_lr_factor = float(params[4])
    reduce_lr_patience = int(params[5])
    reduce_lr_min_lr = float(params[6])
    
    print(f"\nTrying Hyperparameters: LSTM Units = {lstm_units}, Batch Size = {batch_size}, "
          f"Time Steps = {time_steps}, Early Stopping Patience = {early_stopping_patience}, "
          f"Reduce LR Factor = {reduce_lr_factor:.2f}, Reduce LR Patience = {reduce_lr_patience}, "
          f"Min LR = {reduce_lr_min_lr:.6f}")

    # Updating sequence length for data creation
    global X_train, y_train, X_test, y_test
    X, y = create_dataset(model_scaled_features, model_scaled_target, time_steps)
    train_size = int(0.80 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Converting to tensors
    X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
    X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
    y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)
    
    input_shape = (time_steps, X_train.shape[2])
    print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

    # Defining the new model architecture
    model = Sequential([
        # First LSTM layer with regularization
        LSTM(lstm_units, return_sequences=True,
             kernel_regularizer=l2(0.01),
             recurrent_regularizer=l2(0.01),
             input_shape=input_shape),
        Dropout(0.2),

        # Second LSTM layer
        LSTM(lstm_units, return_sequences=False,
             kernel_regularizer=l2(0.01),
             recurrent_regularizer=l2(0.01)),
        Dropout(0.2),

        # Output layer
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=early_stopping_patience,
        restore_best_weights=True
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=reduce_lr_factor,
        patience=reduce_lr_patience,
        min_lr=reduce_lr_min_lr
    )

    # Training the model
    history = model.fit(
        X_train_tensor, y_train_tensor,
        epochs=10,
        batch_size=batch_size,
        validation_data=(X_test_tensor, y_test_tensor),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Getting validation loss
    val_loss = min(history.history['val_loss'])

    print(f"Validation Loss: {val_loss:.6f}")

    return val_loss

# Running PSO with the new model
best_params, best_loss = pso(
    objective_function,
    lb=[b[0] for b in bounds],  # Lower bounds
    ub=[b[1] for b in bounds],  # Upper bounds
    swarmsize=7,               # Number of particles
    maxiter=7,                 # Number of iterations
    debug=True                  # Enable logging
)

print(f"\nBest Hyperparameters: LSTM Units = {int(best_params[0])}, Batch Size = {int(best_params[1])}, "
      f"Time Steps = {int(best_params[2])}, Early Stopping Patience = {int(best_params[3])}, "
      f"Reduce LR Factor = {best_params[4]:.2f}, Reduce LR Patience = {int(best_params[5])}, "
      f"Min LR = {best_params[6]:.6f}")
print(f"Best Validation Loss = {best_loss:.6f}")

No constraints given.

Trying Hyperparameters: LSTM Units = 157, Batch Size = 83, Time Steps = 84, Early Stopping Patience = 12, Reduce LR Factor = 0.20, Reduce LR Patience = 2, Min LR = 0.000029
X_train shape: (12592, 84, 5), X_test shape: (3149, 84, 5)
Epoch 1/10
152/152 [==============================] - 5s 14ms/step - loss: 0.9331 - val_loss: 0.0140 - lr: 0.0010
Epoch 2/10
152/152 [==============================] - 1s 10ms/step - loss: 0.0035 - val_loss: 0.0025 - lr: 0.0010
Epoch 3/10
152/152 [==============================] - 1s 10ms/step - loss: 0.0015 - val_loss: 0.0038 - lr: 0.0010
Epoch 4/10
152/152 [==============================] - 1s 10ms/step - loss: 0.0011 - val_loss: 0.0019 - lr: 0.0010
Epoch 5/10
152/152 [==============================] - 1s 10ms/step - loss: 8.6435e-04 - val_loss: 0.0018 - lr: 0.0010
Epoch 6/10
152/152 [==============================] - 1s 10ms/step - loss: 7.7699e-04 - val_loss: 0.0020 - lr: 0.0010
Epoch 7/10
152/152 [==============================] -

### PSO (4,4)

In [21]:
import random
import numpy as np
import pyswarm
from pyswarm import pso
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

param_ranges = {
    "lstm_units": (32, 256),           # Range for LSTM units
    "batch_size": (16, 128),           # Range for batch size
    "time_steps": (30, 120),           # Range for sequence length
    "early_stopping_patience": (5, 20),  # Range for early stopping patience
    "reduce_lr_factor": (0.1, 0.5),      # Range for reduce_lr factor
    "reduce_lr_patience": (2, 10),       # Range for reduce_lr patience
    "reduce_lr_min_lr": (1e-6, 1e-4)     # Range for minimum learning rate
}

bounds = [
    param_ranges["lstm_units"],            # Bounds for LSTM units
    param_ranges["batch_size"],            # Bounds for batch size
    param_ranges["time_steps"],            # Bounds for sequence length
    param_ranges["early_stopping_patience"],  # Bounds for early stopping patience
    param_ranges["reduce_lr_factor"],         # Bounds for reduce_lr factor
    param_ranges["reduce_lr_patience"],       # Bounds for reduce_lr patience
    param_ranges["reduce_lr_min_lr"]          # Bounds for reduce_lr min_lr
]

def objective_function(params):
    lstm_units = int(params[0])
    batch_size = int(params[1])
    time_steps = int(params[2])
    early_stopping_patience = int(params[3])
    reduce_lr_factor = float(params[4])
    reduce_lr_patience = int(params[5])
    reduce_lr_min_lr = float(params[6])
    
    print(f"\nTrying Hyperparameters: LSTM Units = {lstm_units}, Batch Size = {batch_size}, "
          f"Time Steps = {time_steps}, Early Stopping Patience = {early_stopping_patience}, "
          f"Reduce LR Factor = {reduce_lr_factor:.2f}, Reduce LR Patience = {reduce_lr_patience}, "
          f"Min LR = {reduce_lr_min_lr:.6f}")

    # Updating sequence length for data creation
    global X_train, y_train, X_test, y_test
    X, y = create_dataset(model_scaled_features, model_scaled_target, time_steps)
    train_size = int(0.80 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Converting to tensors
    X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
    X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
    y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)
    
    input_shape = (time_steps, X_train.shape[2])
    print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

    # Defining the new model architecture
    model = Sequential([
        # First LSTM layer with regularization
        LSTM(lstm_units, return_sequences=True,
             kernel_regularizer=l2(0.01),
             recurrent_regularizer=l2(0.01),
             input_shape=input_shape),
        Dropout(0.2),

        # Second LSTM layer
        LSTM(lstm_units, return_sequences=False,
             kernel_regularizer=l2(0.01),
             recurrent_regularizer=l2(0.01)),
        Dropout(0.2),

        # Output layer
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=early_stopping_patience,
        restore_best_weights=True
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=reduce_lr_factor,
        patience=reduce_lr_patience,
        min_lr=reduce_lr_min_lr
    )

    # Training the model
    history = model.fit(
        X_train_tensor, y_train_tensor,
        epochs=10,
        batch_size=batch_size,
        validation_data=(X_test_tensor, y_test_tensor),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Getting validation loss
    val_loss = min(history.history['val_loss'])

    print(f"Validation Loss: {val_loss:.6f}")

    return val_loss

# Running PSO with the new model
best_params, best_loss = pso(
    objective_function,
    lb=[b[0] for b in bounds],  # Lower bounds
    ub=[b[1] for b in bounds],  # Upper bounds
    swarmsize=4,               # Number of particles
    maxiter=4,                 # Number of iterations
    debug=True                  # Enable logging
)

print(f"\nBest Hyperparameters: LSTM Units = {int(best_params[0])}, Batch Size = {int(best_params[1])}, "
      f"Time Steps = {int(best_params[2])}, Early Stopping Patience = {int(best_params[3])}, "
      f"Reduce LR Factor = {best_params[4]:.2f}, Reduce LR Patience = {int(best_params[5])}, "
      f"Min LR = {best_params[6]:.6f}")
print(f"Best Validation Loss = {best_loss:.6f}")

No constraints given.

Trying Hyperparameters: LSTM Units = 177, Batch Size = 115, Time Steps = 95, Early Stopping Patience = 6, Reduce LR Factor = 0.33, Reduce LR Patience = 7, Min LR = 0.000054
X_train shape: (12584, 95, 5), X_test shape: (3146, 95, 5)
Epoch 1/10
110/110 [==============================] - 5s 20ms/step - loss: 1.3624 - val_loss: 0.0400 - lr: 0.0010
Epoch 2/10
110/110 [==============================] - 1s 12ms/step - loss: 0.0095 - val_loss: 0.0048 - lr: 0.0010
Epoch 3/10
110/110 [==============================] - 1s 12ms/step - loss: 0.0020 - val_loss: 0.0033 - lr: 0.0010
Epoch 4/10
110/110 [==============================] - 1s 12ms/step - loss: 0.0013 - val_loss: 0.0017 - lr: 0.0010
Epoch 5/10
110/110 [==============================] - 1s 12ms/step - loss: 0.0010 - val_loss: 0.0017 - lr: 0.0010
Epoch 6/10
110/110 [==============================] - 1s 12ms/step - loss: 8.7710e-04 - val_loss: 0.0022 - lr: 0.0010
Epoch 7/10
110/110 [==============================] - 1s 